In [10]:
import plotly.express as px
import plotly.graph_objects as go
import xarray as xr
from metpy.units import units
import metpy.calc as mpcalc
from dateutil import tz

from datetime import datetime

In [11]:
'''
Define Model Parameters
'''
MODEL_DATE = '20230625'
MODEL_CYCLE = '12z'

# Define area of interest
LAT = 19.0
LON = -83.0

In [12]:
base_url = f'https://nomads.ncep.noaa.gov/dods/gefs/gefs{MODEL_DATE}/gefs_pgrb2ap5_all_{MODEL_CYCLE}'

ds = xr.open_dataset(base_url, decode_times=True)

u_wind = ds.ugrd10m.metpy.sel(lat=LAT, lon=360+LON, method='nearest') * units('m/s')
v_wind = ds.vgrd10m.metpy.sel(lat=LAT, lon=360+LON, method='nearest') * units('m/s')

/Users/rhawthorne/miniconda3/envs/metpy/lib/python3.11/site-packages/xarray/coding/times.py:154: SerializationWarning:

Ambiguous reference date string: 1-1-1 00:00:0.0. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:0.0). To remove this message, remove the ambiguity by padding your reference date strings with zeros.



In [13]:
wind = mpcalc.wind_speed(u_wind, v_wind)
wind = wind.metpy.convert_units('knots')

In [14]:
ens_mean = wind.mean(dim=['ens'])
ens_max = wind.max(dim=['ens'])
p25 = wind.quantile(.25, dim=['ens'])
p75 = wind.quantile(.75, dim=['ens'])

In [15]:
df = wind.to_dataframe('wind')
df.dropna(inplace=True)
df.reset_index(inplace=True)
df

,ens,time,lat,lon,wind
0,1.0,2023-06-25 12:00:00,19.0,277.0,6.992446
1,1.0,2023-06-25 18:00:00,19.0,277.0,7.078246
2,1.0,2023-06-26 00:00:00,19.0,277.0,8.387548
3,1.0,2023-06-26 06:00:00,19.0,277.0,10.108354
4,1.0,2023-06-26 12:00:00,19.0,277.0,8.690406
...,...,...,...,...,...
2010,31.0,2023-07-10 12:00:00,19.0,277.0,14.009378
2011,31.0,2023-07-10 18:00:00,19.0,277.0,12.880319
2012,31.0,2023-07-11 00:00:00,19.0,277.0,11.379419
2013,31.0,2023-07-11 06:00:00,19.0,277.0,15.969101


In [16]:
df_mean = df.groupby(['time'], as_index=False).mean()
df_median = df.groupby(['time'], as_index=False).median()
df_max = df.groupby(['time'], as_index=False).max()
df_p25 = df.groupby(['time'], as_index=False).quantile(.25)
df_p75 = df.groupby(['time'], as_index=False).quantile(.75)

In [17]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['time'], y=df_max['wind'], name='Max'))
fig.add_trace(go.Scatter(x=df['time'], y=df_p75['wind'], name='p75'))
fig.add_trace(go.Scatter(x=df['time'], y=df_mean['wind'], name='Mean'))
fig.add_trace(go.Scatter(x=df['time'], y=df_median['wind'], name='p50'))
fig.add_trace(go.Scatter(x=df['time'], y=df_p25['wind'], name='p25'))

# Set layout options
fig.update_layout(title=f'Wind Speed at {LAT}, {LON}<br>NOAA GFS Ensemble Model',
                  xaxis_title='Time',
                  yaxis_title='Wind Speed (knots)')

# Display the graph
fig.show()

In [18]:
fig.write_image("timeseries-gefs_winds.png", width=1280, height=720, scale=1)